# Введение в DifferentialEquations.jl

## Обыкновенные дифференциальные уравнения

Этот блокнот познакомит вас с diffrentialEquations.jl, предназначенного для решения обыкновенных дифференциальных уравнений (ОДУ). Соответствующей страницей документации является [учебник ODE](http://docs.juliadiffeq.org/latest/tutorials/ode_example.html). Хотя некоторые синтаксисы могут отличаться для других типов уравнений, одни и те же общие принципы имеют место в каждом случае. Наша цель - дать легкое и подробное введение, в котором освещены эти принципы таким образом, чтобы помочь вам обобщить то, что вы узнали.

### Окружение

Если вы новичок в изучении дифференциальных уравнений, может быть полезно прочесть краткую справочную информацию по [определению обыкновенных дифференциальных уравнений](https://ru.wikipedia.org/wiki/Обыкновенное_дифференциальное_уравнение) или совсем [для чайников](http://www.mathprofi.ru/differencialnye_uravnenija_primery_reshenii.html). Мы определяем обыкновенное дифференциальное уравнение как уравнение, которое описывает способ изменения переменной $ u $, то есть 

$$u'= f (u, p, t)$$

где $ p $ - параметры модели, $ t $ - переменная времени, а $ f $ - нелинейная модель того, как изменяется $ u $. Также задача включает в себя информацию о начальном значении: 

$$ u(t_0) = u_0 $$ 

Если вы знаете начальное значение и то, как оно будет меняться со временем, тогда вы знаете, как будет выглядеть система в любой момент времени в будущем. Это интуитивное определение дифференциального уравнения.

### Первая модель: экспоненциальный рост

Нашей первой моделью будет каноническая модель экспоненциального роста. Эта модель говорит, что скорость изменения пропорциональна текущему значению: 

$$u'= au $$ 

где у нас есть начальное значение $ u (0) = u_0 $. Допустим, мы вкладываем 1 доллар в биткойн, который растет со скоростью 0.98 долларов США в год. Затем, задав $ t = 0 $ и измеряя время в годах, получим модель: 

$$ u '= 0.98u $$ 

и $ u (0) = 1.0 $. Задаем функцию в общем виде 

$$ f (u, p, t) = 0.98u $$ 

с $ u_0 = 1,0 $. Если мы хотим решить эту модель за промежуток времени от `t = 0.0` до` t = 1.0`, то мы определяем `ODEProblem`, указав функцию` f`, начальное условие `u0` и промежуток времени:

In [ ]:
using DifferentialEquations
f(u,p,t) = 0.98u
u0 = 1.0
tspan = (0.0,1.0)
prob = ODEProblem(f,u0,tspan)

Для решения нашей `ODEProblem` мы используем команду` solve`.

In [ ]:
sol = solve(prob)

и это все: мы успешно решили наше первое ОДУ! 

#### Анализ решения 

Конечно, тип решения не интересен сам по себе. Мы хотим понять решение! Здесь мы опишем некоторые из основ. Вы можете построить решение, используя рецепт, предоставленный [Plots.jl](http://docs.juliaplots.org/latest/):

In [ ]:
using Plots; gr()
plot(sol)

Из рисунка видно, что решение представляет собой экспоненциальную кривую, которая соответствует нашей интуиции. В качестве рецепта сюжета мы можем аннотировать результат, используя любой из [атрибутов Plots.jl](http://docs.juliaplots.org/latest/attributes/). Например:

In [ ]:
plot(sol,linewidth=5,title="Solution to the linear ODE with a thick line",
     xaxis="Time (in years)",yaxis="u(t) (in \$)",label="My Thick Line!") # legend=false

Используя дополняющую команду `plot!`, мы можем добавить новые графики в уже существующий фрэйм. Для этого ОДУ мы знаем, что истинное решение - это $u (t) = u_0 \exp(at) $, поэтому давайте добавим аналитическое решение к нашему графику:

In [ ]:
plot!(sol.t, t->1.0*exp(0.98t),lw=3,ls=:dash,label="True Solution!")

В предыдущей ячейке использовалась `sol.t`, который захватывает массив моментов времени, из решения:

In [ ]:
sol.t

Аналогично можно получить массив решений `sol.u`:

In [ ]:
sol.u

`sol.u[i]` соответсвующее решение для момента `sol.t[i]`. Мы можем использовать значения из этого массива кортежей:

In [ ]:
[t+u for (u,t) in tuples(sol)]

Однако одной интересной особенностью является то, что по умолчанию решение является непрерывной функцией. Если мы проверим распечатку еще раз:

In [ ]:
sol

Вы можете заметить, что решение имеет интерполяцию 4-го порядка, что означает, что оно является непрерывной функцией точности 4-го порядка. Мы можем назвать решение как функцию времени `sol (t)`. Например, чтобы получить значение при `t = 0,45`, мы можем использовать команду:

In [ ]:
sol(0.45)

#### Контролируя солвер

У diffrentialEquations.jl есть общий набор элементов управления решателем среди его алгоритмов, который можно найти [на странице параметров общего решателя](http://docs.juliadiffeq.org/latest/basics/common_solver_opts.html). Мы подробно опишем некоторые из наиболее широко используемых вариантов. 

Наиболее полезными параметрами являются ключи-команды `abstol` и` reltol`. Они показывают внутреннему адаптивному механизму временного шага, насколько точным является решение, которое вы хотите. Как правило, «reltol» - это относительная точность, а «abstol» - это точность, когда «u» близок к нулю. Эти ключи являются локальными допусками и, следовательно, не являются глобальными переменными. Тем не менее, хорошее эмпирическое правило заключается в том, что общая точность решения на 1-2 цифры меньше относительных допусков. Таким образом, для значений по умолчанию `abstol = 1e-6` и` reltol = 1e-3` можно ожидать общую точность порядка 1-2 цифр. Если мы хотим получить около 6 цифр точности, мы можем использовать команды:

In [ ]:
sol = solve(prob,abstol=1e-8,reltol=1e-8)

Теперь мы не видим ощутимых отличий от истинного решения:

In [ ]:
plot(sol)
plot!(sol.t, t->1.0*exp(0.98t),lw=3,ls=:dash,label="True Solution!")

Обратите внимание, что при уменьшении погрешности число шагов, которые должен был сделать решатель, составило `9` вместо предыдущего` 5`. Между точностью и скоростью существует компромисс, и вы сами должны определить, какой баланс вам подходит. 

Другой распространенный вариант - использовать `saveat`, чтобы программа сохранения сохранялась в определенные моменты времени. Например, если мы хотим получить решение при четной сетке `t = 0.1k` для целых чисел` k`, мы бы использовали команду:

In [ ]:
sol = solve(prob,saveat=0.1)

Обратите внимание, что когда используется `saveat`, непрерывные выходные переменные больше не сохраняются, и, таким образом,интерполяция `sol(t)`, имеет только первый порядок. Мы можем сохранить неравномерную сетку точек, передав набор значений в `saveat`. Например:

In [ ]:
sol = solve(prob,saveat=[0.2,0.7,0.9])

По умолчанию он всегда сохраняет первое и последнее значения, но мы также можем отключить это:

In [ ]:
sol = solve(prob,saveat=[0.2,0.7,0.9],save_start = false, save_end = false)

Если нам нужна экономия памяти, мы также можем отключить непрерывный вывод напрямую с помощью `density = false`:

In [ ]:
sol = solve(prob,dense=false)

и чтобы отключить все промежуточные сохранения, мы можем использовать `save_everystep = false`:

In [ ]:
sol = solve(prob,save_everystep=false)

Более сложные способы сохранения, такие как сохранение функционалов решения, обрабатываются с помощью `SavingCallback` в [Callback Library](http://docs.juliadiffeq.org/latest/features/callback_library.html#SavingCallback-1), которая будет раскрыта позже в учебнике.

####  Выбор алгоритмов решения

Не существует лучшего алгоритма для численного решения дифференциального уравнения. Когда вы вызываете `solve(prob)`, DifferentialEquations.jl делает предположение о хорошем алгоритме для вашей задачи, учитывая свойства, которые вы запрашиваете (допуски, информация о сохранении и т.д.). Однако во многих случаях вам может потребоваться более прямой контроль. Более поздняя записная книжка поможет представить различные *алгоритмы* в DifferentialEquations.jl, но сейчас давайте введем *синтаксис*. 

Наиболее важным определяющим фактором при выборе численного метода является жесткость модели. Жесткость грубо характеризуется якобианом `f` с большими собственными значениями. Это довольно математично, и мы можем думать об этом более интуитивно: если у вас есть большие числа в `f` (например, параметры порядка` 1e5`), то это, вероятно, жестко. Или, как создатель пакета MATLAB ODE, Лоуренс Шампин, любит определять его: если стандартные алгоритмы медленные, то они жесткие. Мы углубимся в диагностику жесткости в следующем уроке, но сейчас учтите, что если вы считаете, что ваша модель может быть жесткой, вы можете подсказать это для выбора алгоритма с помощью `alg_hints = [:stiff]`.

In [ ]:
sol = solve(prob,alg_hints=[:stiff])

Жесткие алгоритмы должны решать неявные уравнения и линейные системы на каждом этапе, поэтому они должны использоваться только при необходимости. 

Если мы хотим выбрать алгоритм напрямую, вы можете передать тип алгоритма после задачи как «solve(prob, alg)». Например, давайте решим эту проблему, используя алгоритм `Tsit5()`, и просто для галочки давайте изменим относительный допуск на `1e-6` одновременно:

In [ ]:
sol = solve(prob,Tsit5(),reltol=1e-6)

### Системы ОДУ: уравнение Лоренца

Теперь давайте перейдем к системе ODE. [Уравнение Лоренца](https://ru.wikipedia.org/wiki/Аттрактор_Лоренца) является знаменитым «аттрактором бабочки», породившим теорию хаоса. Определяется системой ОДУ:

$$ \frac{dx}{dt} = \sigma (y - x) $$
$$ \frac{dy}{dt} = x (\rho - z) -y $$
$$ \frac{dz}{dt} = xy - \beta z $$

Чтобы определить систему дифференциальных уравнений в DifferentialEquations.jl, мы определим нашу `f` как вектор-функцию с векторным начальным условием. Таким образом, для вектора `u = [x, y, z]` мы имеем функцию:

In [ ]:
function lorenz!(du,u,p,t)
    σ,ρ,β = p
    du[1] = σ*(u[2]-u[1])
    du[2] = u[1]*(ρ-u[3]) - u[2]
    du[3] = u[1]*u[2] - β*u[3]    
end

Обратите внимание, что здесь мы использовали формат *in-place*, который записывает вывод в предварительно выделенный вектор `du`. Для систем уравнений формат выделения на месте быстрее. Мы используем начальное условие $ u_0 = [1.0,0.0,0.0] $ следующим образом:

In [ ]:
u0 = [1.0,0.0,0.0]

Наконец, для этой модели мы использовали параметры `p`. Нам нужно также установить это значение в `ODEProblem`. Для нашей модели мы хотим решить, используя параметры $ \sigma = 10 $, $ \rho = 28 $ и $ \beta = 8/3 $, и, таким образом, мы строим коллекцию параметров:

In [ ]:
p = (10,28,8/3) # we could also make this an array, or any other type!

Теперь мы генерируем тип ODEProblem задав параметры явно. Давайте решим это дело на промежутке времени от «t = 0» до «t = 100»:

In [ ]:
tspan = (0.0,100.0)
prob = ODEProblem(lorenz!,u0,tspan,p)

Теперь, как и прежде, мы решаем проблему:

In [ ]:
sol = solve(prob)

В этом случае применяются те же функции обработки решения. Таким образом, sol.t хранит временные точки, а sol.u представляет собой массив, хранящий решение в соответствующих временных точках. 

Однако есть несколько дополнительных функций, которые полезно знать при работе с системами уравнений. Прежде всего, `sol` также действует как массив. `sol[i]` возвращает решение в i-й момент времени.

In [ ]:
sol.t[10], sol[10]

Кроме того, решение действует как матрица, где `sol [j, i]` - значение переменной `j` в момент времени` i`:

In [ ]:
sol[2,10]

Мы можем получить реальную матрицу, выполнив преобразование:

In [ ]:
A = convert(Array,sol)

Это то же самое, что и sol, то есть `sol [i, j] = A [i, j]`, но теперь это истинная матрица. На графике по умолчанию будут показаны временные ряды для каждой переменной:

In [ ]:
plot(sol)

Если вместо этого мы хотим построить значения друг против друга, мы можем использовать команду `vars`. Давайте построим переменную `1` против переменной` 2` против переменной `3`:

In [ ]:
plot(sol,vars=(1,2,3))

Это классический график аттрактора Лоренца, где ось `x` равна` u [1] `, ось` y` равна `u [2]`, а ось `z` равна` u [3] `. Обратите внимание, что график по умолчанию использует интерполяцию, но мы можем это отключить:

In [ ]:
plot(sol,vars=(1,2,3),denseplot=false)

Кхе!.. Это показывает, как вычисление непрерывного решения сэкономило много ресурсов, вычисляя только разреженное решение и заполняя значения! Обратите внимание, что в vars `0 = время`, и, таким образом, мы можем построить временные ряды одного компонента, например:

In [ ]:
plot(sol,vars=(0,2))

### DSL для параметризованных функций

Во многих случаях вы можете определять множество функций с параметрами. Существует домен-специфический язык (DSL), определяемый макросом `ode_def` помгоющий в решении этой распространенной проблемы. Например, мы можем определить уравнение Лотки-Вольтерра:

$$ \frac{dx}{dt} = ax - bxy $$
$$ \frac{dy}{dt} = -cy + dxy $$

как:

In [ ]:
function lotka_volterra!(du,u,p,t)
  du[1] = p[1]*u[1] - p[2]*u[1]*u[2]
  du[2] = -p[3]*u[2] + p[4]*u[1]*u[2]
end

В этих масивах легко запутаться, что решается макросом `ode_def`:

In [ ]:
]add ParameterizedFunctions

In [ ]:
using ParameterizedFunctions

In [ ]:
lv! = @ode_def LotkaVolterra begin
  dx = a*x - b*x*y
  dy = -c*y + d*x*y
end a b c d

Затем мы можем использовать результат так же, как и функцию ODE:

In [ ]:
u0 = [1.0,1.0]
p = (1.5,1.0,3.0,1.0)
tspan = (0.0,10.0)
prob = ODEProblem(lv!,u0,tspan,p)
sol = solve(prob)
plot(sol)

Мало того, что синтаксис DSL до жути удобен, так он еще делает всякую магию за кулисами. Например, в следующих частях учебника будет описано, как решатели для жестких дифференциальных уравнений должны использовать якобиан в вычислениях. Здесь DSL использует символическое дифференцирование для автоматического получения этой функции:

In [ ]:
lv!.Jex

DSL может получать много других функций; эта способность используется для ускорения решателей. Расширение diffrentialEquations.jl, [Latexify.jl](https://korsbo.github.io/Latexify.jl/latest/tutorials/parameterizedfunctions.html), позволяет извлекать эти фрагменты как выражения LaTeX.

## Внутренние типы

Последняя базовая функция пользовательского интерфейса - выбор типов. DiffrentialEquations.jl учитывает ваши типы ввода для определения используемых внутренних типов. Таким образом, поскольку в предыдущих случаях, когда мы использовали значения `Float64` для начального условия, это означало, что внутренние значения будут решаться с использованием` Float64`. Мы убедились, что время указано через значения `Float64`, а это означает, что временные шаги также будут использовать 64-разрядные числа с плавающей точкой. Но этим можно не ограничиваться. 

В качестве короткого примера, скажем, мы хотим решить ODE, определяемую матрицей. Для этого мы можем просто использовать матрицу в качестве входных данных.

In [ ]:
A  = [1. 0  0 -5
      4 -2  4 -3
     -4  0  0  1
      5 -2  2  3]
u0 = rand(4,2)
tspan = (0.0,1.0)
f(u,p,t) = A*u
prob = ODEProblem(f,u0,tspan)
sol = solve(prob)

Нет реального отличия от того, что мы делали раньше, но теперь в этом случае `u0` является матрицей` 4x2`. Из-за этого решением в каждый момент времени является матрица:

In [ ]:
sol[3]

В DifferentialEquations.jl вы можете использовать любой тип, для которого определены `+`, `-`,` * `,` / `и имеет соответствующую` norm`. Например, если мы хотим получить числа с плавающей запятой произвольной точности, мы можем изменить входные данные на матрицу BigFloat:

In [ ]:
big_u0 = big.(u0)

и мы можем решить `ODEProblem` с числами произвольной точности, используя это начальное условие:

In [ ]:
prob = ODEProblem(f,big_u0,tspan)
sol = solve(prob) # теееееерпееение...

In [ ]:
sol[1,3]

Чтобы действительно использовать всё это, мы хотели бы изменить `abstol` и` reltol` на маленькие значения! Обратите внимание, что тип для «времени» отличается от типа для зависимых переменных, и это может быть использовано для оптимизации алгоритма путем сохранения нескольких значений точности. Мы также можем преобразовать время в произвольную точность, определив наш промежуток времени с помощью переменных BigFloat:

In [ ]:
prob = ODEProblem(f,big_u0,big.(tspan))
sol = solve(prob)

Давайте закончим, показывая более сложное использование типов. Для небольших массивов обычно быстрее выполнять операции над статическими массивами с помощью пакета [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl). Синтаксис похож на синтаксис обычных массивов, но для этих специальных массивов мы используем макрос `
SMatrix`, чтобы указать, что мы хотим создать статический массив.

In [ ]:
]add StaticArrays

In [ ]:
using StaticArrays
A  = @SMatrix [ 1.0  0.0 0.0 -5.0
                4.0 -2.0 4.0 -3.0
               -4.0  0.0 0.0  1.0
                5.0 -2.0 2.0  3.0]
u0 = @SMatrix rand(4,2)
tspan = (0.0,1.0)
f(u,p,t) = A*u
prob = ODEProblem(f,u0,tspan)
sol = solve(prob)

In [ ]:
sol[3]

## Заключение 

Это основные элементы управления в diffrentialEquations.jl. Все уравнения определяются с помощью типа задачи, и команда `solve` используется с выбором алгоритма (или по умолчанию), чтобы получить решение. Каждое решение действует одинаково, как массив `sol[i]` с `sol.t[i]`, а также как непрерывная функция `sol(t)` с хорошей командой plot `plot(sol)`. Общие параметры решателя могут использоваться для управления для любого типа уравнения. Наконец, типы, используемые при численном решении, определяются типами ввода, и это может использоваться для решения с произвольной точностью и добавления дополнительных оптимизаций (это можно использовать, например, для решения с помощью графических процессоров!). Хотя всё было показано на ОДУ, эти методы обобщаются и на другие типы уравнений.